In [10]:
import pandas as pd
#load the dataset
file_path='recipe_final (1).csv'
recipe_df=pd.read_csv(file_path)
recipe_df.head(3)

,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,calories,fat,carbohydrates,protein,cholesterol,sodium,fiber,ingredients_list
0,0,222388,Homemade Bacon,5.00,https://images.media-allrecipes.com/userphotos...,3,15,36,1,42,21,81,2,"['pork belly', 'smoked paprika', 'kosher salt'..."
1,1,240488,"Pork Loin, Apples, and Sauerkraut",4.76,https://images.media-allrecipes.com/userphotos...,29,19,18,10,73,33,104,41,"['sauerkraut drained', 'Granny Smith apples sl..."
2,2,218939,Foolproof Rosemary Chicken Wings,4.57,https://images.media-allrecipes.com/userphotos...,12,17,36,2,48,24,31,4,"['chicken wings', 'sprigs rosemary', 'head gar..."


In [11]:
recipe_df.isnull().sum()

Unnamed: 0          0
recipe_id           0
recipe_name         0
aver_rate           0
image_url           0
review_nums         0
calories            0
fat                 0
carbohydrates       0
protein             0
cholesterol         0
sodium              0
fiber               0
ingredients_list    0
dtype: int64

In [12]:
recipe_df['ingredients_list'][0]


"['pork belly', 'smoked paprika', 'kosher salt', 'ground black pepper']"

In [30]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
vectorizer=TfidfVectorizer()
X_ingredients = vectorizer.fit_transform(recipe_df['ingredients_list'])

In [35]:
scaler = StandardScaler()
X_numerical = scaler.fit_transform(recipe_df[['calories','fat','carbohydrates','protein','cholesterol','sodium','fiber']] ) 

In [36]:
X_numerical

array([[-0.1317045 ,  0.46001924, -1.15482863, ...,  0.04256474,
         1.13990476, -0.76393724],
       [ 0.23857551, -0.33625589, -0.01920347, ...,  0.52871248,
         1.59202345,  2.53220175],
       [ 0.05343551,  0.46001924, -1.02864806, ...,  0.16410167,
         0.15703804, -0.59490447],
       ...,
       [-0.77969453, -0.82286847, -0.27156462, ..., -0.68665688,
        -0.39336732, -0.25683894],
       [ 0.33114552,  0.32730672,  0.73787996, ...,  0.20461398,
        -0.31473798, -0.51038809],
       [-1.33511455, -0.9998185 , -1.02864806, ..., -0.80819381,
        -0.13782197, -0.42587171]])

In [37]:
X_combined = np.hstack([X_numerical,X_ingredients.toarray()])
X_combined

array([[-0.1317045 ,  0.46001924, -1.15482863, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.23857551, -0.33625589, -0.01920347, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.05343551,  0.46001924, -1.02864806, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.77969453, -0.82286847, -0.27156462, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.33114552,  0.32730672,  0.73787996, ...,  0.        ,
         0.        ,  0.        ],
       [-1.33511455, -0.9998185 , -1.02864806, ...,  0.        ,
         0.        ,  0.        ]])

In [38]:
knn=NearestNeighbors(n_neighbors=3,metric='euclidean')
knn.fit(X_combined)

NearestNeighbors(metric='euclidean', n_neighbors=3)

In [44]:
def recommend_recipes(input_features):
    input_features_scaled=scaler.transform([input_features[:7]])
    input_ingredients_tranformed = vectorizer.transform([input_features[7]])
    input_combined = np.hstack([input_features_scaled,input_ingredients_tranformed.toarray()])
    distances,indices=knn.kneighbors(input_combined)
    recommendations=recipe_df.iloc[indices[0]]
    return recommendations[['recipe_name','ingredients_list','image_url']]

input_features = [15,36,1,42,21,81,2,'pork belly,smoked paprika,kosher salt']
recommendations = recommend_recipes(input_features)
recommendations
    

C:\Users\Jorden\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,recipe_name,ingredients_list,image_url
0,Homemade Bacon,"['pork belly', 'smoked paprika', 'kosher salt'...",https://images.media-allrecipes.com/userphotos...
3507,Char Siu (Chinese Barbeque Pork),"['honey', 'Shaoxing cooking wine', 'hoisin sau...",https://images.media-allrecipes.com/userphotos...
1926,Adobo Twist,"['vegetable oil', 'onion', 'head garlic', 'por...",https://images.media-allrecipes.com/userphotos...
